In [47]:
import os
import pandas as pd
import re
from functools import reduce

# Set base folders
energy_data_folder = 'STAT 390 Project/Energy Data'
states = [
    'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA',
    'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME',
    'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM',
    'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX',
    'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'
]

# =========================
# Load ATB Data
# =========================

import os
import pandas as pd

def load_atb():
    atb_folder = os.path.join(energy_data_folder, 'ATB')
    years = [f for f in os.listdir(atb_folder) if f.isdigit()]
    latest_year = max(years)
    atb_path = os.path.join(atb_folder, latest_year)

    csv_files = [f for f in os.listdir(atb_path) if f.endswith('.csv')]
    if not csv_files:
        raise ValueError(f"❌ No CSV files found in {atb_path}")

    atb_file = os.path.join(atb_path, csv_files[0])
    print(f"✅ Loading ATB file: {atb_file}")

    atb_df = pd.read_csv(atb_file, low_memory=False, index_col=0).reset_index(drop=True)

    # Confirm year column
    first_col = atb_df.columns[0]
    if 'year' not in first_col.lower():
        raise ValueError(f"❌ First column '{first_col}' does not contain 'year'.")

    # Step 1: Remove 'default' column (no longer needed)
    atb_df = atb_df.drop(columns=['default'])

    # Step 2: Create final metadata dict per row
    metadata_rows = []
    for _, row in atb_df.iterrows():
        year = row[first_col]
        metadata = row.drop(first_col).to_dict()

        metadata_rows.append({
            'year': int(year),
            'state': 'US',
            'metadata': {'ATB': metadata}
        })

    # Step 3: Build final DataFrame
    final_atb_df = pd.DataFrame(metadata_rows)

    print("✅ Final ATB metadata dataframe ready!")
    return final_atb_df

# =========================
# Load RECS Data
# =========================

def load_recs():
    recs_folder = os.path.join(energy_data_folder, 'RECS')
    years = [f for f in os.listdir(recs_folder) if f.isdigit()]
    latest_year = max(years)
    saved_year = int(latest_year)
    recs_path = os.path.join(recs_folder, latest_year)

    recs_files = [f for f in os.listdir(recs_path) if f.endswith('.csv') and 'recs' in f.lower()]
    version_files = []
    for f in recs_files:
        match = re.search(r'v(\d+)', f.lower())
        if match:
            version_files.append((int(match.group(1)), f))
    if not version_files:
        raise ValueError(f"❌ No RECS files with version found in {recs_path}")

    highest_version_file = max(version_files, key=lambda x: x[0])[1]
    recs_file = os.path.join(recs_path, highest_version_file)
    print(f"✅ Loading RECS file: {recs_file}")

    # Load RECS file
    recs_df = pd.read_csv(recs_file, low_memory=False)

    if 'state_postal' not in recs_df.columns:
        raise ValueError("❌ 'state_postal' column missing in RECS file")

    # Rename for clarity
    recs_df = recs_df.rename(columns={'state_postal': 'state'})

    # Step 1: Create metadata dictionaries
    metadata_rows = []
    for _, row in recs_df.iterrows():
        state = row['state']
        metadata = row.drop('state').to_dict()

        # Overwrite/add year manually to be safe
        metadata['year'] = saved_year

        metadata_rows.append({
            'year': saved_year,
            'state': state,
            'metadata': {'RECS': metadata}
        })

    # Step 2: Build final DataFrame
    final_recs_df = pd.DataFrame(metadata_rows)

    print("✅ Final RECS metadata dataframe ready!")
    return final_recs_df

# =========================
# Load RMI Operations Emissions
# =========================

def load_rmi():
    # --- Setup ---
    rmi_folder = os.path.join(energy_data_folder, 'RMI')

    # Find operations file
    operations_files = [f for f in os.listdir(rmi_folder) if 'operations_emissions_by_fuel' in f.lower()]
    if not operations_files:
        raise ValueError("❌ No RMI operations_emissions_by_fuel file found.")

    operations_path = os.path.join(rmi_folder, operations_files[0])
    print(f"✅ Loading RMI Operations file: {operations_path}")

    # --- Load the file ---
    operations_df = pd.read_csv(operations_path, dtype=str)

    # --- Minimal cleaning ---
    numeric_columns = operations_df.columns[-5:]  # last 5 numeric columns
    keep_columns = ['year', 'state', 'fuel_type_category'] + list(numeric_columns)

    operations_clean = operations_df[keep_columns].copy()

    # Convert numeric columns
    for col in numeric_columns:
        operations_clean[col] = pd.to_numeric(operations_clean[col], errors='coerce')

    # --- Group by year, state, fuel ---
    grouped = operations_clean.groupby(['year', 'state', 'fuel_type_category'], as_index=False).sum()

    # --- Pivot into wide format ---
    reshaped_list = []
    for metric in numeric_columns:
        pivot = grouped.pivot_table(
            index=['year', 'state'],
            columns='fuel_type_category',
            values=metric,
            aggfunc='sum'
        )
        pivot = pivot.add_prefix(f'{metric}_')
        reshaped_list.append(pivot)

    operations_wide = pd.concat(reshaped_list, axis=1).reset_index()
    operations_wide.columns.name = None  # remove extra axis name

    # --- Build metadata column ---
    def pack_metadata(row):
        return {'RMI': {col: row[col] for col in operations_wide.columns if col not in ['year', 'state']}}

    final_df = pd.DataFrame({
        'year': operations_wide['year'].astype(int),
        'state': operations_wide['state'],
        'metadata': operations_wide.apply(pack_metadata, axis=1)
    })

    print(f"✅ Final RMI operations metadata dataframe ready!")
    return final_df


# =========================
# Load Total Energy Data
# =========================

def load_total_energy():
    total_folder = os.path.join(energy_data_folder, 'Total Energy/2025')
    files = [f for f in os.listdir(total_folder) if f.endswith('.csv') and f != 'total_energy_contents.csv']
    all_dfs = []
    
    for file_name in files:
        file_path = os.path.join(total_folder, file_name)
    
        try:
            # Quick scan to detect if 'YYYYMM' format appears (in 3rd row)
            preview = pd.read_csv(file_path, nrows=5, header=None, dtype=str)
            header_preview = preview.iloc[2].fillna('').tolist()
    
            first_col = header_preview[0]
    
            if 'yyyymm' in str(first_col).lower():
                # ✅ State-organized special case
    
                # Step 1: Read with header at 3rd row (skip first 2 rows)
                df = pd.read_csv(file_path, skiprows=2, dtype=str)
    
                # Step 2: Clean header
                df.columns = [col.replace(',', '').strip() if isinstance(col, str) else col for col in df.columns]
    
                # Step 3: Drop 'US' column if it exists
                df = df.drop(columns=['US'], errors='ignore')
    
                # Step 4: Reformat 'YYYYMM' to 'YYYY'
                original_year_col = df.columns[0]
                df[original_year_col] = df[original_year_col].str[:4]
                df = df.rename(columns={original_year_col: 'year'})
    
                # Step 5: Melt into long format
                value_col_name = file_name.replace('.csv', '').lower()
                df_melted = df.melt(id_vars=['year'], var_name='state', value_name=value_col_name)
    
                # Step 6: Convert values to numeric
                df_melted[value_col_name] = pd.to_numeric(df_melted[value_col_name], errors='coerce')
    
                # Step 7: Group by (year, state) and take sum
                df_grouped = df_melted.groupby(['year', 'state'], as_index=True).sum()
    
                # Step 8: Sort (optional polish)
                df_grouped = df_grouped.sort_index()
    
                # Add to the list
                all_dfs.append(df_grouped)
    
                print(f"✅ Processed file: {file_name}")
    
            else:
                # ✅ Normal file case
    
                # Step 1: Read header rows manually
                df_header1 = pd.read_csv(file_path, skiprows=8, nrows=1, header=None, dtype=str)
                df_header2 = pd.read_csv(file_path, skiprows=9, nrows=1, header=None, dtype=str)
    
                # Step 2: Combine headers
                combined_headers = []
                for h1, h2 in zip(df_header1.iloc[0], df_header2.iloc[0]):
                    h1_clean = str(h1).strip().replace(',', '').replace(' ', '_').lower()
                    h2_clean = str(h2).strip().replace(',', '').replace(' ', '_').lower()
                    if h2_clean and h2_clean.lower() != 'nan':
                        combined_headers.append(f"{h1_clean}_{h2_clean}")
                    else:
                        combined_headers.append(h1_clean)
    
                # Step 3: Read full data
                df = pd.read_csv(file_path, skiprows=10, names=combined_headers, dtype=str)
    
                # Step 4: Clean column names
                col_a_original = combined_headers[0]
    
                # Step 5: Rename first column to 'year'
                df = df.rename(columns={col_a_original: 'year'})
    
                # Step 6: Rename other columns properly
                cleaned_columns = {}
                for col in df.columns:
                    if col != 'year':
                        cleaned_columns[col] = f"{file_name.replace('.csv', '').lower()}_{col}_{col_a_original}"
                df = df.rename(columns=cleaned_columns)
    
                df['state'] = 'US'
                df = df.set_index(['year', 'state'])
    
                all_dfs.append(df)
    
                print(f"✅ Processed file: {file_name}")
    
        except Exception as e:
            print(f"❌ Failed to process {file_name}: {e}")
    
    # Merge all DataFrames efficiently
    if all_dfs:
        final_df = reduce(lambda left, right: left.join(right, how='outer'), all_dfs)
        final_df = final_df.sort_index()
    
        print("✅ Final Total Energy dataframe ready!")
    
    else:
        final_df = pd.DataFrame()
        print("❌ No data was loaded.")

    return final_df

# =========================
# Load SEDS Data
# =========================

def load_seds():
    seds_folder = os.path.join(energy_data_folder, 'SEDS')
    years = [f for f in os.listdir(seds_folder) if f.isdigit()]
    latest_year = max(years)
    seds_path = os.path.join(seds_folder, latest_year)

    csv_files = [f for f in os.listdir(seds_path) if f.endswith('.csv') and 'complete' in f.lower()]
    if not csv_files:
        raise ValueError(f"❌ No 'complete' CSV files found in {seds_path}")

    seds_file = os.path.join(seds_path, csv_files[0])
    print(f"✅ Loading SEDS file: {seds_file}")

    seds_df = pd.read_csv(seds_file, low_memory=False)

    if 'Data_Status' in seds_df.columns:
        seds_df = seds_df.drop(columns=['Data_Status'])

    seds_pivoted = seds_df.pivot_table(index=['Year', 'StateCode'], columns='MSN', values='Data').reset_index()
    seds_pivoted.columns.name = None
    seds_pivoted = seds_pivoted.rename(columns={'Year': 'year', 'StateCode': 'state'})
    final_seds_df = seds_pivoted.set_index(['year', 'state'])

    print("✅ Final SEDS dataframe ready!")
    return final_seds_df

# =========================
# Merge Total Energy & SEDS
# =========================

def merge_seds_and_total(seds_df, total_df):
    # 🛠 Step 1: Standardize both datasets
    def standardize_index(df):
        if isinstance(df.index, pd.MultiIndex):
            df = df.reset_index()
        else:
            df = df.reset_index()

        if 'year' not in df.columns or 'state' not in df.columns:
            raise ValueError("❌ DataFrame missing 'year' or 'state' for merging.")

        df['year'] = df['year'].astype(int)
        df['state'] = df['state'].astype(str)

        return df.set_index(['year', 'state'])

    seds_std = standardize_index(seds_df)
    total_std = standardize_index(total_df)

    # 🛠 Step 2: Merge SEDS and Total Energy
    merged = seds_std.join(total_std, how='outer')

    # 🛠 Step 3: Reset index (now year and state are columns)
    merged = merged.reset_index()

    # 🛠 Step 4: Build metadata column
    priority_cols = ['year', 'state']
    data_cols = [col for col in merged.columns if col not in priority_cols]

    def row_to_metadata(row):
        metadata = {col: row[col] for col in data_cols if pd.notna(row[col])}
        return {"SEDS_AND_TOTAL_ENERGY": metadata}

    merged['metadata'] = merged.apply(row_to_metadata, axis=1)

    # 🛠 Step 5: Keep only year, state, metadata
    final_df = merged[['year', 'state', 'metadata']]

    print(f"✅ Merged SEDS and Total Energy it metadata dataframe")
    return final_df

In [67]:
from functools import reduce
import pandas as pd
import numpy as np

def final_concat():
    # 🛠 Step 1: Concatenate all datasets
    combined_df = pd.concat([
        atb_df,
        recs_df,
        rmi_df,
        seds_and_total_df
    ], axis=0, ignore_index=True)
    
    # 🗺 Step 2: State abbreviation → full state name
    # Reference: https://www.usps.com/send/official-abbreviations.htm
    state_abbr_to_name = {
        'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas',
        'CA': 'California', 'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware',
        'FL': 'Florida', 'GA': 'Georgia', 'HI': 'Hawaii', 'ID': 'Idaho',
        'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa', 'KS': 'Kansas',
        'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
        'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi',
        'MO': 'Missouri', 'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada',
        'NH': 'New Hampshire', 'NJ': 'New Jersey', 'NM': 'New Mexico', 'NY': 'New York',
        'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio', 'OK': 'Oklahoma',
        'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
        'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah',
        'VT': 'Vermont', 'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia',
        'WI': 'Wisconsin', 'WY': 'Wyoming', 'DC': 'District of Columbia'
    }
    
    # Only replace if it matches a valid abbreviation
    combined_df['state'] = combined_df['state'].apply(
        lambda x: state_abbr_to_name.get(x, x)  # If x is a valid abbreviation, replace; else keep as is
    )
    
    # 🧹 Step 3: Optional sort
    combined_df = combined_df.sort_values(['year', 'state']).reset_index(drop=True)
    
    # ✅ Done
    print(f"✅ Final combined dataset shape: {combined_df.shape}")
    return combined_df


In [51]:
# =========================
# Run all data loads
# =========================

atb_df = load_atb()
recs_df = load_recs()
rmi_df = load_rmi()

seds_df = load_seds()
total_df = load_total_energy()
seds_and_total_df = merge_seds_and_total(seds_df, total_df)


✅ Loading RMI Operations file: STAT 390 Project/Energy Data\RMI\operations_emissions_by_fuel_2024.csv
✅ Final RMI operations metadata dataframe ready!
✅ Loading SEDS file: STAT 390 Project/Energy Data\SEDS\2022\Complete_SEDS.csv
✅ Final SEDS dataframe ready!
✅ Processed file: total_energy_approximate_heat_rates_for_electricity.csv
✅ Processed file: total_energy_average_prices_of_electricity_to_ultimate_customers.csv
✅ Processed file: total_energy_biodiesel_overview.csv
✅ Processed file: total_energy_capacity_factors_and_usage_factors_at_electric_generators_commercial_sector.csv
✅ Processed file: total_energy_capacity_factors_and_usage_factors_at_electric_generators_electric_power_sector.csv
✅ Processed file: total_energy_capacity_factors_and_usage_factors_at_electric_generators_industrial_sector.csv
✅ Processed file: total_energy_capacity_factors_and_usage_factors_at_electric_generators_total_all_sectors.csv
✅ Processed file: total_energy_carbon_dioxide_emissions_from_energy_consumptio

In [69]:
final_energy_df = final_concat()

✅ Final combined dataset shape: (595242, 3)


In [73]:
final_energy_df

,year,state,metadata
0,1949,US,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_appro...
1,1950,US,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_appro...
2,1951,US,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_appro...
3,1952,US,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_appro...
4,1953,US,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_appro...
...,...,...,...
595237,2025,Virginia,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_dcfc_...
595238,2025,Washington,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_dcfc_...
595239,2025,West Virginia,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_dcfc_...
595240,2025,Wisconsin,{'SEDS_AND_TOTAL_ENERGY': {'total_energy_dcfc_...


In [71]:
# 📂 Save the final_energy_df to a CSV file
final_csv_path = 'STAT 390 Project/Energy Data/final_energy_df.csv'

# Save to CSV
final_energy_df.to_csv('STAT 390 Project/Energy Data/final_energy_df.csv')

print(f"✅ Saved final_energy_df to {final_csv_path}")

✅ Saved final_energy_df to STAT 390 Project/Energy Data/final_energy_df.csv
